In [74]:
import torch

In [75]:
import datetime

import gym

In [76]:
from time import sleep
from tqdm import tqdm
from torch import nn
import copy
import random
from torch.utils.tensorboard import SummaryWriter

# class : Config
- 학습에 필요한 각종 세팅, replay memory 등을 저장 및 관리

In [89]:
class Config():
    def __init__(self, lr = 0.01,
                gamma = 0.6,
                min_epsilon = 0.1,
                renew_target = 100,
                store_and_replay = True,
                double_dqn = True):
        self.lr = lr
        self.gamma = gamma
        self.min_epsilon = min_epsilon
        self.store_and_replay = store_and_replay
        self.double_dqn = double_dqn
        self.max_time_steps = 1000
        self.renew_target = renew_target
        
        self.replay_memory_fail = []
        self.replay_memory_fail_size = 1000 # 실패 state 의 결과를 1000개까지만 저장한다.
        self.replay_memory_success = []
        
        self.prev_success = set() # 디버깅용 : 손님을 태워 목적지에 도착한상태를 저장
        
        dt = datetime.datetime.now() ## 기록 title 설정
        dt_str = dt.strftime("%Y-%m-%d %H:%M:%S")
        self.opt_str = f"SR_{store_and_replay}_DD_{double_dqn}_LOSS"
        self.writer = SummaryWriter(f"runs/{self.opt_str}_{dt_str}")
        
        self.num_episodes = 5000
    
    def get_epsilon(self, episode = -1):
        if episode < 0:
            return self.min_epsilon
        return max(self.min_epsilon, 0.99**episode)
    
    '''
    success, fail 을 따로 저장하도록 사이즈 조정을 해줌.
    '''
    def _insert_record(self, memory, rec, memory_size = -1):
        if rec in memory:
            #print(f'in memory. {rec}')
            return
        if len(memory) < memory_size or memory_size == -1:
            memory.append(rec)
        else:
            memory[random.randint(0, memory_size-1)] = rec
    
    def insert_record(self, rec):
        done = rec[-1]
        if done:
            self._insert_record(self.replay_memory_success, rec)
            self.prev_success.add(rec[0])
        else:
            self._insert_record(self.replay_memory_fail, rec, self.replay_memory_fail_size)
            
    def get_replay_memory(self):
        return self.replay_memory_success + self.replay_memory_fail
    
    def get_replay_record(self):
        replay_memory = self.get_replay_memory()
        mid_replay = random.randint(0, len(replay_memory) - 1)
        return mid_replay, replay_memory[mid_replay]

# class : QNet
- DQN

In [78]:
class QNet(nn.Module):
    def __init__(self, num_states, num_actions, hidden_dim = 16):
        super().__init__()
        
        self.layers = nn.Sequential(
            nn.Embedding(num_states, 2*hidden_dim),
            nn.Linear(2*hidden_dim, 2*hidden_dim),
            nn.PReLU(),
            nn.Linear(2*hidden_dim, hidden_dim),
            nn.PReLU(),
            nn.Linear(hidden_dim, num_actions)
        )
        
    def forward(self, x):
        x = self.layers(x)
        return x

# def : train
- 모델 학습 (전체 에피소드에 대해)

In [79]:
def train(env, config, qnet):
    optimizer = torch.optim.SGD(qnet.parameters(), lr = 0.001)
    if config.double_dqn:
        qnet2 = copy.deepcopy(qnet)
    else:
        qnet2 = qnet
        
    criteria = nn.MSELoss()
    num_episodes = config.num_episodes
    
    for i in tqdm(range(1, num_episodes+1)):
        rtn_loss = train_episode(env, config, qnet, qnet2, optimizer, criteria, episode_count = i) # 수정
        if i % config.renew_target == 0:
            if config.double_dqn:
                qnet2 = copy.deepcopy(qnet)
        
        if i == 1 or i%50 == 0:
            test(env, config, qnet, i, rtn_loss) # 수정
            
    print("Training finished.\n")

# def : train_episode
- 모델 학습 (한 에피소드)

In [80]:
def train_episode(env, config, qnet, qnet2, optimizer, criteria, episode_count = -1):
    done = False
    n_steps = 0
    state = env.reset()
    tot_reward = 0
    loss = 0
    penalties = 0
    
    while not done and n_steps < config.max_time_steps:
        #n_steps += 1
        state_t = torch.LongTensor([state])
        
        epsilon = config.get_epsilon(episode_count)
        #print("Epsilon", epsilon)
        
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample()
        else:
            with torch.no_grad():
                q_hat = qnet(state_t)
                action = torch.argmax(q_hat[0]).item()
                
        next_state, reward, done, info = env.step(action)
        
        tot_reward += reward
        
        new_tuple = (state, action, next_state, reward, done)
        
        state = next_state
        if reward == -10:
            penalties += 1
            
        # Replay
        if config.store_and_replay:
            config.insert_record(new_tuple)
            loss_i, steps_i = replay(qnet, qnet2, config, optimizer, criteria)
        else:
            loss_i, steps_i = replay(qnet, qnet2, config, optimizer, criteria, rec = new_tuple)
        loss += loss_i
        n_steps += 1
    

    return loss #추가

# def : replay
- 모델 학습 (한 에피소드)

In [81]:
def replay(qnet, qnet2, config, optimizer, criteria, num_instances = 5, rec = None):
    loss_i = 0
    
    for _ in range(num_instances):
        optimizer.zero_grad()
        if config.store_and_replay:
            mid_replay, rec = config.get_replay_record()
            
        state, action, next_state, reward, done = rec
        
        # Set target value
        if done:
            y_t = torch.Tensor([reward])
        else:
            next_state_r_t = torch.LongTensor([next_state])
            with torch.no_grad():
                q_next = qnet2(next_state_r_t)
                y_t = reward + config.gamma * q_next.max(dim=-1)[0]
                
        # Make a prediction
        state_r_t = torch.LongTensor([state])
        q_hat = qnet(state_r_t)
        q_hat = q_hat[:, action]
        
        # Update
        loss = criteria(q_hat, y_t)
        loss.backward()
        optimizer.step()
        loss_i += loss.item()
        
    return loss_i, num_instances

# def : test
- 테스트

In [82]:
def test(env, config, qnet, global_step = -1, loss = -1): # 수정
    qnet.eval()
    total_epochs, total_penalites = 0, 0
    episodes = 100
    
    total_reward = 0
    writer = config.writer
    
    for _ in tqdm(range(episodes)):
        state = env.reset()
        epochs, penalties, reward = 0, 0, 0
        
        done = False
        
        while not done and epochs < config.max_time_steps:
            with torch.no_grad():
                state_t = torch.LongTensor([state])
                q_hat = qnet(state_t)
                action = torch.argmax(q_hat[0]).item()
                
            state, reward, done, info = env.step(action)
            total_reward += reward
            if reward == -10:
                penalties += 1
                
            epochs += 1
            
        total_penalites += penalties
        total_epochs += epochs
        
    avg_steps = total_epochs / episodes
    avg_penalty = total_penalites / episodes
    avg_reward = total_reward / episodes
    
    print(f"Result after {episodes} episodes:")
    print(f"Average timesteps per episodes: {avg_steps}")
    print(f"Average penalty per episodes: {avg_penalty}")
    print(f"Average reward per episodes: {avg_reward}")
    
    if global_step > 0:
        writer.add_scalar("Steps", avg_steps, global_step)
        writer.add_scalar("Penalty", avg_penalty, global_step)
        writer.add_scalar("Reward", avg_reward, global_step)
        writer.add_scalar("Loss", loss, global_step) # 추가
          

# 메인

In [91]:
env = gym.make("Taxi-v3").env

print("Action Space {}".format(env.action_space))
print("State Space {}".format(env.observation_space))
store_and_replay = True
double_dqn = False

print(f"Store & replay: {store_and_replay}")
print(f"Doublde DQN: {double_dqn}")

qnet = QNet(env.observation_space.n, env.action_space.n, hidden_dim = 32)
config = Config(store_and_replay = store_and_replay, double_dqn = double_dqn, lr = 0.01, gamma = 0.7, renew_target = 20)

train(env, config, qnet)
test(env, config, qnet, config.num_episodes)

Action Space Discrete(6)
State Space Discrete(500)
Store & replay: True
Doublde DQN: False


  0%|                                       | 1/5000 [00:08<12:27:10,  8.97s/it]

Result after 100 episodes:
Average timesteps per episodes: 1000.0
Average penalty per episodes: 0.0
Average reward per episodes: -1000.0


  1%|▍                                      | 50/5000 [01:38<5:20:17,  3.88s/it]

Result after 100 episodes:
Average timesteps per episodes: 990.07
Average penalty per episodes: 0.0
Average reward per episodes: -989.86


...

Result after 100 episodes:
Average timesteps per episodes: 259.02
Average penalty per episodes: 0.0
Average reward per episodes: -243.27


100%|███████████████████████████████████████| 5000/5000 [21:41<00:00,  3.84it/s]


Result after 100 episodes:
Average timesteps per episodes: 288.82
Average penalty per episodes: 0.0
Average reward per episodes: -273.7
Training finished.



100%|█████████████████████████████████████████| 100/100 [00:01<00:00, 67.58it/s]

Result after 100 episodes:
Average timesteps per episodes: 219.48
Average penalty per episodes: 0.0
Average reward per episodes: -202.89


In [84]:
print(env.observation_space.n, env.action_space.n)

500 6
